In [6]:
input_data={'CreditScore': 619,
 'Gender': 'Female',
 'Age': 42,
 'Tenure': 2,
 'Balance': 0.0,
 'NumOfProducts': 1,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'Exited': 1,
 'Geography':'France'}

In [7]:
import pandas as pd

In [8]:
input_df=pd.DataFrame(data=[input_data])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography
0,619,Female,42,2,0.0,1,1,1,1,France


In [9]:
# Export models
import os
root_dir=os.path.dirname(os.getcwd())
preprocessor_dir='preprocessor'

label_encoder_obj_file_path=os.path.join(root_dir,preprocessor_dir,'label_encoder.pkl')
onehotencoder_obj_file_path=os.path.join(root_dir,preprocessor_dir,'onehot_encoder.pkl')
scaler_obj_file_path=os.path.join(root_dir,preprocessor_dir,'scaler.pkl')

model_dir='model'
model_file_path=os.path.join(root_dir,model_dir,'model.h5')

## Load the ANN model
import tensorflow
from tensorflow.keras.models import load_model

model=load_model(model_file_path)

import pickle
## Label Encoder
with open (file=label_encoder_obj_file_path,mode='rb') as file:
    label_encoder_obj=pickle.load(file=file)


## OneHotEncoder
with open(file=onehotencoder_obj_file_path,mode='rb') as file:
    onehot_encoder_obj=pickle.load(file=file)


## Standardization
with open (file=scaler_obj_file_path,mode='rb') as file:
    scaler_obj=pickle.load(file=file)   

In [10]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography
0,619,Female,42,2,0.0,1,1,1,1,France


In [12]:
# Apply Labelencoder on Gender
input_df['Gender']=label_encoder_obj.transform(input_df['Gender'])

In [13]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography
0,619,0,42,2,0.0,1,1,1,1,France


In [14]:
# Apply OHE ON Geography
geo_encoded=onehot_encoder_obj.transform(input_df[['Geography']]).toarray()
geo_encoded

array([[1., 0., 0.]])

In [16]:
geo_encoded_df=pd.DataFrame(data=geo_encoded,
             columns=onehot_encoder_obj.get_feature_names_out(input_features=['Geography']),dtype='int')
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1,0,0


In [17]:
input_df=pd.concat(objs=[input_df,geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,1,France,1,0,0


In [19]:
# Drop geography column
input_df.drop(columns='Geography',inplace=True)

In [20]:
# Standardization
input_scaled=scaler_obj.transform(input_df)

In [21]:
input_scaled

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  1.96626042,  1.00150113,
        -0.57946723, -0.57638802]])

In [22]:
#prediction
model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


array([[92939.79]], dtype=float32)

In [27]:
model.predict(input_scaled)[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


np.float32(92939.79)